# Golomb Self Describing Sequence
The Golomb's self-describing sequence $G(n)$ is the only nondecreasing sequence of natural numbers such that $n$ appears exactly $G(n)$ times in the sequence. The first few terms of the sequence $n$ are:
$
1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 6
$

You are given $G(10^3) = 86$ and $G(10^6) = 6137$. 

You are also give $\sum_{i=1}^{10^3} G(i^3) = 153506976$. 

Find $\sum_{i=1}^{10^6} G(i^3)$. 

In [1]:
def generate_n_golomb(n):
    out = [1, 2]
    curr = 2
    used = 1
    idx = 2

    while idx < n:
        if used < out[curr-1]:
            out.append(curr)
            used += 1
        else:
            curr += 1
            used = 1
            out.append(curr)
        idx += 1
    return out

In [21]:
from functools import lru_cache

def rec_nth_golomb(n):
    @lru_cache(int(10e6))
    def rec(n):
        if n == 1: 
            return 1
        return 1 + rec(n -rec(rec(n-1)))
    return rec(n)

rec_nth_golomb(1e8)

RecursionError: maximum recursion depth exceeded in comparison

In [2]:
def golomb_dp(n):
    n = int(n)
    if n == 1:
        return 1

    # Create a list to store Golomb sequence values
    golomb_seq = [0] * (n + 1)
    golomb_seq[1] = 1

    for i in range(2, n + 1):
        golomb_seq[i] = 1 + golomb_seq[i - golomb_seq[golomb_seq[i - 1]]]

    return golomb_seq[n]

In [13]:
import timeit

def timed_g():
    return generate_n_golomb(1e4)

timeit.timeit(timed_g, number=1000)/1000

0.000581604872000753

In [37]:
generate_n_golomb(5)

[1, 2, 2, 3, 3]

In [3]:
a = generate_n_golomb(1e8+1)

In [2]:
threshold = 1e18

In [6]:
a[-1]*105729>threshold

False

In [5]:
# Products[i] = Golomb[1]*1 + Golomb[2]*2 + ... + Golomb[i]*i
def calc_prod(i, a: list):
    idx = 1
    tot = 0
    while idx<i:
        tot += a[idx-1]*idx
        idx += 1
    return tot    

In [29]:
products = []
curr = 0
idx = 0
while curr < threshold+1:
    curr += a[idx] * (idx+1)
    products.append(curr)
    idx += 1

I know that products[i] tells me that Golomb[products[i]] appears at least i-times in the sequence; for instance products[4] = 11 means that all G[x>11] appear at least 4 times in the sequence.

To know G[i=10] I know it's between products[2]=5 and products[3]=11; I know that any number 6 <= x <= 11 appears 3 times in the sequence.

So I know that:
$
G[1]*1 + G[2]*2 = 5 \\
G[1]*1 + G[2]*2 + G[3]*3 = 11 \\
$
and:
$
G[1] + G[2] = 3 \\
G[1] + G[2] + G[3] = 5 \\
$


$
G[1] + G[2] = 3 \\
G[1] + G[2] + G[3] = 5 \\
G[1] + G[2] + G[3] + G[4] = 8 \\
G[1] + G[2] + G[3] + G[4] + G[5] = 11
$



In [37]:
products[:10]

[0, 1, 5, 11, 23, 38, 62, 90, 122, 167]

In [43]:
import math

def golomb_sequence(limit):
    """
    Generate the Golomb sequence up to the given limit.
    Returns the Golomb sequence, cumulative sums, and cumulative products.
    """
    golomb = [0, 1]  # G(0) is unused, G(1) = 1
    sums = [0, 1]    # Cumulative sums
    products = [0, 1]  # Cumulative weighted sums (i * G(i))
    
    # Compute the sequence until products exceed the cubic limit
    cubic_limit = limit ** 3
    while products[-1] < cubic_limit:
        i = len(golomb)
        g = 1 + golomb[i - golomb[golomb[i - 1]]]
        golomb.append(g)
        sums.append(sums[-1] + g)
        products.append(products[-1] + g * i)
        if i % 1e6 == 0:
            print(i, g, sums[-1], products[-1])
    
    return golomb, sums, products

def find_g_n_cubed(n_cubed, products, sums):
    """
    Find G(n^3) using linear interpolation.
    """
    # Find the interval for n^3
    index = 1
    # Use binary search to find the interval for n^3
    low, high = 0, len(products) - 1
    while low < high:
        mid = (low + high) // 2
        if products[mid] < n_cubed:
            low = mid + 1
        else:
            high = mid
    index = low

    # Linear interpolation
    from_products = products[index - 1]
    to_products = products[index]
    ratio = (n_cubed - from_products) / (to_products - from_products)

    from_sums = sums[index - 1]
    to_sums = sums[index]
    g_n_cubed = from_sums + math.ceil(ratio * (to_sums - from_sums))
    
    return g_n_cubed

def sum_g_n_cubed(limit):
    """
    Compute the sum of G(n^3) for 1 <= n < limit.
    """
    golomb, sums, products = golomb_sequence(limit)
    total_sum = 0

    for n in range(1, limit):
        n_cubed = n ** 3
        g_n_cubed = find_g_n_cubed(n_cubed, products, sums)
        total_sum += g_n_cubed

    return total_sum

# Input limit (1 <= n < 10^6)
limit = 10**6
result = sum_g_n_cubed(limit)
print(result)


1000000 6137 3792638403 2344023456102415
2000000 9419 11642479607 14391201349694894
3000000 12102 22437898167 41603009912867973
4000000 14458 35739453276 88354593378726651
5000000 16596 51281472859 158471803016495259
6000000 18576 68878895033 255422576996674145
7000000 20433 88392497852 382415739373324908
8000000 22191 109712161761 542459723141481692
9000000 23867 132747742086 738401818567438811
10000000 25474 157423553071 972955874693962102
56098610614277014


In [8]:
a[int(1e9)]

438744

In [19]:
a[100:130]

[21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24]

In [28]:
golden = (1 + 5 ** 0.5) / 2

In [43]:
golomb_dp(10e7)

TypeError: can't multiply sequence by non-int of type 'float'

In [16]:
tot = 0
for i in range(1, 1000):
    tot += a[int(i**3)-1]

In [17]:
tot

153506976